In [3]:
# beat the drum 
# wrote by Jinxin Dong 
# 2018. 11. 15
# latest amended 2018.11.16
from OpenGL.GL import *
from OpenGL.GLU import *
from OpenGL.GLUT import *

In [ ]:
import numpy as np
import cv2
import pygame
import time
import glob
import math
import sys
#import graphics

class Playdrum:
    def __init__(self):
        # area threshold, used to judge whether the drum been hit or not
        self.area_lim =1000
        self.area_lim2 = 300
        pygame.init()
        # game start time 
        self.start_time = time.time()
        self.first_num = 0
        self.second_num = 0
        self.mask_h = 100
        self.mask_w = 100
        self.take_out = 0
        self.hit_on_count = 0
        self.hit_side_count = 0
        # record the hit time (on drum)
        self.p_time = time.time()
        self.n_time = time.time()
        # record the hit time (drum side)
        self.p_side_time = time.time()
        self.n_side_time = time.time()
        
    def play_sound(self,sound_path):
    # TODO: play the music 
        drum_beats = pygame.mixer.Sound(sound_path)
        drum_beats.play()
        #print ('playsound!')
        
    def binarize(self,gray_in, threshold=200):
        #ret,thresh1 = cv2.threshold(img,threshold,255,cv.THRESH_BINARY)
        binary_image = np.where(gray_in<threshold,0,255).astype('uint8')
        return binary_image

    def detect_stick_on(self,image_cut):
    # TODO: detcet the stick hit directly on the drum 
        cv2.imshow('stick on',image_cut)
        image, cnt, hierarchy = cv2.findContours(image_cut.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        area_list = []
        for i in range(len(cnt)):
            area_list.append(cv2.contourArea(cnt[i]))
        max_area = max(area_list)
        if max_area>4000:
            max_area = 0
        print (max_area)
        if max_area>self.area_lim:
            self.n_time = time.time()
            self.hit_on_count  = self.hit_on_count+1
            time_diff = self.n_time - self.p_time
            #print (time_diff)
            #print (self.hit_on_count)
            if time_diff> 0.4:
                if self.hit_on_count > 1:
                    self.play_sound("Ka.wav")
                    self.hit_on_count = 0
                    self.p_time = time.time()
        return image_cut
    
    def detect_stick_side(self,image_cut):
    # TODO: detect the stick hit the drum side
    
        image_cut = image_cut.astype('uint8')
        cv2.imshow('stick side',image_cut.astype('uint8'))
        image, cnt, hierarchy = cv2.findContours(image_cut.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        area_list = []
        for i in range(len(cnt)):
            area_list.append(cv2.contourArea(cnt[i]))
        max_area = max(area_list)
        if max_area>5000:
            max_area = 0
        print (max_area)
        if max_area>self.area_lim2:
            self.n_side_time = time.time()
            self.hit_side_count  = self.hit_side_count+1
            time_diff = self.n_side_time - self.p_side_time
            #print (time_diff)
            print (self.hit_side_count)
            if time_diff> 0.2:
                if self.hit_side_count == 1:
                    self.play_sound("Dong.wav")
                    #self.hit_side_count = 0
                    self.p_side_time = time.time()
                elif self.hit_side_count > 1:
                    self.hit_side_count = 0
        return image_cut
    
    def moment_opencv(self,labelled_in):
        # create object moment 
        res = {}
        for label_idx in np.unique(labelled_in):
            if label_idx == 0:
                continue
            masked_in = (labelled_in==label_idx).astype(labelled_in.dtype)
            moments = cv2.moments(masked_in)
            res[label_idx] = [moments['m00'],moments['m01'],moments['m10'],
                              moments['m02'],moments['m11'],moments['m20'],
                              moments['mu02'],moments['mu11'],moments['mu20']]
        return res
    def createCircularMask(self,h, w, center=None, radius=None):
        # create the mask with a empty circle inside 
        if center is None:
            center = [int(w/2), int(h/2)]
        if radius is None:
            radius = min(center[0], center[1], w-center[0], h-center[1])

        Y, X = np.ogrid[:h, :w]
        dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

        mask = dist_from_center <= radius
        return mask
    
    def run(self):
        # run main fuction 
        cap = cv2.VideoCapture(1)
        count = 0
        prievous_thresh_original = 0
        prievous_thresh_cut = 0
        scaler = 1.2
        count = 1
        # create the mask
        mask =  self.createCircularMask(self.mask_h, self.mask_w, center=(self.mask_h/2,self.mask_w/2), radius=45)
        mask = np.array(abs(1-mask))
        #cv2.imshow('frame11',mask.astype('uint8'))
        #print (type(mask))
        #print (mask.shape)
        #print (mask)
        
        while(True):
            try:
                
                ret, frame = cap.read()
                output = frame
                gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
                # get the drum's location
                circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1.2, 100)
                
                # ensure at least some circles were found
                if circles is not None:
                    # convert the (x, y) coordinates and radius of the circles to integers
                    circles = np.round(circles[0, :]).astype("int")

                    # loop over the (x, y) coordinates and radius of the circles
                    for (x, y, r) in circles:
                        # draw the circle in the output image, then draw a rectangle
                        # corresponding to the center of the circle
                        cv2.circle(output, (x, y), r, (0, 255, 0), 4)
                        cv2.rectangle(output, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1)
                        par = np.int64(scaler*r)
                        retval, thresh = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)
                        thresh_cut = cv2.resize(thresh[y-par:y+par,x-par:x+par],(self.mask_h,self.mask_w))
                        thresh_cut_inv = cv2.bitwise_not(thresh_cut)
                        thresh_original = cv2.resize(thresh[y-r:y+r,x-r:x+r],(self.mask_h,self.mask_w))
                        #cv2.imshow('frame4',thresh_cut)

                        if count !=1:
                            thresh1 = self.detect_stick_side((thresh_cut-previous_thresh_cut)*mask)
                            #thresh1 = self.detect_stick_side(thresh_cut)
                            thresh2 = self.detect_stick_on(thresh_original-previous_thresh_original)

                        #cv2.imshow('frame3',thresh_original)
                        #cv2.imshow('frame1', thresh)
                        previous_thresh_cut = thresh_cut 
                        previous_thresh_original = thresh_original
                        count = count+1
            except:
                pass

            cv2.imshow('frame', output)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break 
        cap.release()
        cv2.destroyAllWindows()
        
if __name__ == '__main__':
    p = Playdrum()
    p.run()

28.0
2.0
7.5
11.5
21.5
11.5
44.5
4.5
313.5
1
364.0
333.0
2
378.0
28.0
55.0
55.0
14.5
1862.0
3
0
1636.0
4
0
222.5
503.5
105.0
268.0
102.0
131.0
815.0
5
626.0
982.0
1
1114.5
1302.0
2
1712.5
328.5
3
1486.5
1059.0
4
1261.0
997.0
5
1682.5
457.0
1
3691.5
720.5
2
2169.0
1050.5
3
1637.5
460.5
4
3611.5
1059.5
5
1709.5
626.0
1
2988.0
1574.5
2
1305.5
1541.5
1
1006.5
1559.5
2
1976.5
1559.5
3
2470.5
1559.5
4
2924.0
1559.5
5
2491.0
2645.0
6
1804.5
1559.5
1
1812.0
1559.5
2
1790.0
1559.5
3
2109.5
1559.5
4
2098.5
1521.5
5
1006.5
2625.5
1
1346.0
2629.5
2
1006.5
1559.5
3
2051.5
2617.5
4
1006.5
1559.5
5
2110.5
2669.5
6
2073.0
1559.5
1
1817.0
2654.5
2
1006.5
1559.5
3
2141.5
1559.5
4
2204.0
1559.5
5
2171.5
1559.5
6
2179.0
1109.5
7
1006.5
1559.5
1
2390.5
1559.5
2
3366.0
1559.5
3
2649.5
1559.5
4
2686.5
1559.5
5
3240.5
1559.5
6
3284.0
1559.5
7
2092.0
1559.5
8
1820.0
1559.5
1
2637.0
1559.5
2
2093.5
1559.5
3
2126.0
1559.5
4
3216.0
1559.5
5
0
151.0
432.0
68.0
588.0
30.0
240.5
110.5
257.5
187.0
259.5
127.0
26.0
85

1.0
18.0
1.0
27.0
15.0
39.5
1.0
30.0
4.0
99.0
2.0
9.0
1.0
20.5
4.0
28.0
0.0
16.0
0.0
2.5
19.0
90.0
506.5
2
316.5
287.0
191.0
401.5
1
379.0
22.5
71.0
16.0
76.0
16.5
103.0
25.5
81.0
72.0
94.5
78.0
186.5
402.0
2
150.0
0
970.0
3026.0
1
1005.0
2513.0
2
471.0
142.5
65.5
3.0
31.5
3.0
28.5
0.0
9.5
0.0
2.0
0.0
21.5
14.0
103.0
19.0
76.0
38.0
78.5
7.0
84.5
18.0
73.0
33.0
149.5
29.0
254.0
0.0
50.0
1.0
29.5
5.0
105.5
0.0
73.0
312.0
3
238.5
224.0
152.5
54.0
55.5
8.5
38.5
9.0
97.5
10.0
114.0
2.0
30.0
15.0
90.0
28.5
213.5
19.0
135.0
44.0
59.5
14.0
36.0
15.0
19.5
2.5
15.0
13.5
77.5
69.0
69.0
26.0
51.0
17.0
91.0
0.0
8.5
0.0
106.0
2.0
69.5
0.0
21.0
3.0
21.0
0.0
20.5
0.0
2.0
0.0
12.5
0.0
6.5
0.0
9.5
0.0
20.5
148.5
31.5
75.0
29.5
15.0
67.5
1919.5
1
970.5
1873.5
2
453.0
35.0
65.0
31.0
40.5
8.0
88.5
206.5
270.0
71.0
110.0
274.0
178.5
17.5
57.5
1.0
36.0
16.0
84.0
5.0
77.0
6.0
38.5
9.0
46.5
0.0
3.5
0.0
28.0
2.0
55.0
1.0
19.5
2.0
19.0
3.0
42.0
18.0
50.5
0.0
2.5
4.0
30.0
5.0
32.0
83.5
37.5
488.0
3
398.0
520.0
1


In [ ]:
# old method 
import numpy as np
import cv2
import pygame

import time


class Playdrum:
    def __init__(self):
        self.area_lim = 61000
        self.area_lim2 = 200
        pygame.init()
        self.start_time = time.time()
        
    def play_sound(self,sound_path):
        drum_beats = pygame.mixer.Sound(sound_path)
        drum_beats.play()
        print ('playsound!')
    def binarize(self,gray_in, threshold=200):
        #ret,thresh1 = cv2.threshold(img,threshold,255,cv.THRESH_BINARY)
        binary_image = np.where(gray_in<threshold,0,255).astype('uint8')
        return binary_image

    def detect_stick(self,image_cut):
        thresh = self.binarize(image_cut,128)
        labeled_image = self.sequential_label(thresh)
        area_lim = 61000
        area_lim2 = 500
        cv2.imshow('frame11', thresh)
        res = self.moment_opencv(labeled_image)
        list_A = []
        for i in res:
            list_A.append(res[i][0])

        if list_A:
            #print ('No stick detected!')
            
            if max(list_A)<self.area_lim:
                #print (max(list_A))
                self.play_sound("Ka.wav")

            #print (list_A)
            list_A.pop(list_A.index(max(list_A)))
            if list_A:
                #print ('No stick detected!')
                if max(list_A)>self.area_lim2:
                    #print (max(list_A))
                    self.play_sound("Dong.wav")

        return thresh
    def sequential_label(self,binary_in):
        a = binary_in.shape
        a = np.array([int(v) for v in a])
        I = binary_in
        label = 0
        labeled_image = np.zeros((a[0],a[1]))
        record = []
        for i in range(1,a[0]):
            for j in range(1,a[1]):
                if I[i][j]!=0: 
                    if labeled_image[i-1][j-1]!=0:
                        labeled_image[i][j] = labeled_image[i-1][j-1]

                    elif labeled_image[i][j-1]==0 and labeled_image[i-1][j]==0:
                        label = label+1
                        labeled_image[i][j] = label
                    elif labeled_image[i-1][j]!=0 and labeled_image[i][j-1]==0:
                        labeled_image[i][j] = labeled_image[i-1][j]

                    elif labeled_image[i-1][j]==0 and labeled_image[i][j-1]!=0:
                        labeled_image[i][j] = labeled_image[i][j-1]    

                    elif labeled_image[i-1][j]!=0 and labeled_image[i][j-1]!=0:

                        if labeled_image[i][j-1]==labeled_image[i-1][j]:
                            labeled_image[i][j] = labeled_image[i][j-1]

                        else: 
                            minValue = min(labeled_image[i][j-1],labeled_image[i-1][j])
                            maxValue = max(labeled_image[i][j-1],labeled_image[i-1][j])
                            labeled_image[i][j] = labeled_image[i][j-1]
                            # record equivalence of labels
                            labeled_image = np.where(labeled_image == minValue, maxValue, labeled_image)

        labelled_img = labeled_image
        # normalized

        max_labelled_img = labelled_img.max()
        labelled_img = (labelled_img*(255/max_labelled_img)).astype('uint8')


        return labelled_img
    def moment_opencv(self,labelled_in):
        res = {}
        for label_idx in np.unique(labelled_in):
            if label_idx == 0:
                continue
            masked_in = (labelled_in==label_idx).astype(labelled_in.dtype)
            moments = cv2.moments(masked_in)
            res[label_idx] = [moments['m00'],moments['m01'],moments['m10'],
                              moments['m02'],moments['m11'],moments['m20'],
                              moments['mu02'],moments['mu11'],moments['mu20']]
        return res

    
    def run(self):
        cap = cv2.VideoCapture(0)
        count = 0
        scaler = 1.2
        while(True):
            ret, frame = cap.read()
            output = frame
            gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
            circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1.2, 100)

            # ensure at least some circles were found
            if circles is not None:
                # convert the (x, y) coordinates and radius of the circles to integers
                circles = np.round(circles[0, :]).astype("int")

                # loop over the (x, y) coordinates and radius of the circles
                for (x, y, r) in circles:
                    # draw the circle in the output image, then draw a rectangle
                    # corresponding to the center of the circle
                    cv2.circle(output, (x, y), r, (0, 255, 0), 4)
                    cv2.rectangle(output, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1)
                    par = np.int64(scaler*r)
                    image_cut = gray[y-par:y+par,x-par:x+par]
                    

                    thresh = self.detect_stick(image_cut)
                    #cv2.imshow('frame1', thresh)

            cv2.imshow('frame', output)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break 
        cap.release()
        cv2.destroyAllWindows()
        
if __name__ == '__main__':
    p = Playdrum()
    p.run()

In [ ]:
#  backup 
import numpy as np
import cv2
import pygame
import time


class Playdrum:
    def __init__(self):
        self.area_lim =1000
        
        self.area_lim2 = 1000
        pygame.init()
        self.start_time = time.time()
        self.first_num = 0
        self.second_num = 0
        self.mask_h = 100
        self.mask_w = 100
        self.take_out = 0
        self.hit_on_count = 0
        self.hit_side_count = 0
        self.p_time = time.time()
        self.n_time = time.time()
        
    def play_sound(self,sound_path):
        drum_beats = pygame.mixer.Sound(sound_path)
        drum_beats.play()
        #print ('playsound!')
        
    def binarize(self,gray_in, threshold=200):
        #ret,thresh1 = cv2.threshold(img,threshold,255,cv.THRESH_BINARY)
        binary_image = np.where(gray_in<threshold,0,255).astype('uint8')
        return binary_image

    def detect_stick_on(self,image_cut):
        cv2.imshow('frame_',image_cut)
        image, cnt, hierarchy = cv2.findContours(image_cut.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        area_list = []
        for i in range(len(cnt)):
            area_list.append(cv2.contourArea(cnt[i]))
        max_area = max(area_list)
        #print (max_area)
        if max_area>self.area_lim:
            self.n_time = time.time()
            self.hit_on_count  = self.hit_on_count+1
            time_diff = self.n_time - self.p_time
            print (time_diff)
            print (self.hit_on_count)
            if time_diff> 0.3:
                if self.hit_on_count > 2:
                    self.play_sound("drumbeats/drum.wav")
                    self.hit_on_count = 0
                    self.p_time = time.time()
        return image_cut
    
    def detect_stick_side(self,image_cut):
        image_cut = image_cut.astype('uint8')
        cv2.imshow('frame_',image_cut.astype('uint8'))
        image, cnt, hierarchy = cv2.findContours(image_cut.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        area_list = []
        for i in range(len(cnt)):
            area_list.append(cv2.contourArea(cnt[i]))
        max_area = max(area_list)
        #print (max_area)
        if max_area>self.area_lim:
            self.n_time = time.time()
            self.hit_on_count  = self.hit_on_count+1
            time_diff = self.n_time - self.p_time
            print (time_diff)
            print (self.hit_on_count)
            if time_diff> 0.3:
                if self.hit_on_count > 2:
                    self.play_sound("drumbeats/drum.wav")
                    self.hit_on_count = 0
                    self.p_time = time.time()
        return image_cut
    
    def moment_opencv(self,labelled_in):
        res = {}
        for label_idx in np.unique(labelled_in):
            if label_idx == 0:
                continue
            masked_in = (labelled_in==label_idx).astype(labelled_in.dtype)
            moments = cv2.moments(masked_in)
            res[label_idx] = [moments['m00'],moments['m01'],moments['m10'],
                              moments['m02'],moments['m11'],moments['m20'],
                              moments['mu02'],moments['mu11'],moments['mu20']]
        return res
    def createCircularMask(self,h, w, center=None, radius=None):

        if center is None:
            center = [int(w/2), int(h/2)]
        if radius is None:
            radius = min(center[0], center[1], w-center[0], h-center[1])

        Y, X = np.ogrid[:h, :w]
        dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

        mask = dist_from_center <= radius
        return mask
    
    def run(self):
        cap = cv2.VideoCapture(0)
        count = 0
        prievous_thresh_original = 0
        prievous_thresh_cut = 0
        scaler = 1.2
        count = 1
        mask =  self.createCircularMask(self.mask_h, self.mask_w, center=(self.mask_h/2,self.mask_w/2), radius=40)
        
        mask = np.array(abs(1-mask))
        cv2.imshow('frame11',mask.astype('uint8'))
        print (type(mask))
        print (mask.shape)
        print (mask)
        while(True):
            
            ret, frame = cap.read()
            output = frame
            gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
            circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1.2, 100)
            
            # ensure at least some circles were found
            if circles is not None:
                # convert the (x, y) coordinates and radius of the circles to integers
                circles = np.round(circles[0, :]).astype("int")

                # loop over the (x, y) coordinates and radius of the circles
                for (x, y, r) in circles:
                    # draw the circle in the output image, then draw a rectangle
                    # corresponding to the center of the circle
                    cv2.circle(output, (x, y), r, (0, 255, 0), 4)
                    cv2.rectangle(output, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1)
                    par = np.int64(scaler*r)
                    retval, thresh = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)
                    thresh_cut = cv2.resize(thresh[y-par:y+par,x-par:x+par],(self.mask_h,self.mask_w))
                    thresh_cut_inv = cv2.bitwise_not(thresh_cut)
                    thresh_original = cv2.resize(thresh[y-r:y+r,x-r:x+r],(self.mask_h,self.mask_w))
                    #cv2.imshow('frame4',thresh_cut)
                    
                    if count !=1:
                        thresh1 = self.detect_stick_side((thresh_cut-previous_thresh_cut)*mask)
                        #thresh1 = self.detect_stick_side(thresh_cut)
                        #thresh2 = self.detect_stick_on(thresh_original-previous_thresh_original)

                    #cv2.imshow('frame3',thresh_original)
                    #cv2.imshow('frame1', thresh)
                    previous_thresh_cut = thresh_cut 
                    previous_thresh_original = thresh_original
                    count = count+1

            cv2.imshow('frame', output)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break 
        cap.release()
        cv2.destroyAllWindows()
        
if __name__ == '__main__':
    p = Playdrum()
    p.run()